<a href="https://colab.research.google.com/github/falconlee236/DeepLearningFrom_Scratch/blob/main/Book_1/ch07/Chapter_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7 convolutional neural network(CNN)

**7.4 Implementing Convolution/Pooling layer**

*7.4.1 4-dimension Array*

In [ ]:
import numpy as np

# data 10, channel 1 height 28, width 28
x = np.random.rand(10, 1, 28, 28)
x.shape

In [ ]:
x[0].shape

In [ ]:
x[1].shape

In [ ]:
x[0, 0] # or x[0][0]

*7.4.3 Implementing Convolutional layer*

In [ ]:
%cd drive/MyDrive/DeepLearningFrom_Scratch/Book_1/ch07/

In [ ]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7) # (data, channel, height, width)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75) -> column =  3 * 5 * 5 number of filter elements

x2 = np.random.rand(10, 3, 7, 7) # 10 data
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (9, 75)

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2 * self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2 * self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # filter deployment ->750 elements, reshpae(10, -1) -> 10 * 75 array
        out = np.dot(col, col_W) + self.b

        out = out.reshpae(N, out_h, out_w, -1).trasnpose(0, 3, 1, 2)

        return out


*7.4.4 Implementing Pooling Layer*

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # deployment (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)

        # max value (2)
        out = np.max(col, axis=1)

        # reshape (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

**7.5. Implementing CNN**

In [ ]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_ouput_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_ouput_size = int(filter_num * (conv_ouput_size/2) * (conv_ouput_size/2)) # 2 * 2 pooling

        self.params = {}
        self.params['W1'] = weight_init_std * np.rand.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.rand.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.rand.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_2=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        # forwarding
        self.loss(x, t)

        # backwarding 
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for laayer in layers:
            dout = layer.backward(dout)

        # result save
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db

        return grads

In [ ]:
#ch07/train_convnet.py

